In [1]:
#Installation

!pip install asf_search sentinelhub rasterio geopandas shapely numpy scipy scikit-image matplotlib folium earthaccess --quiet
!pip install rioxarray xarray netCDF4 --quiet

In [2]:
#Importing all necessary libraries

import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Geospatial libraries
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.merge import merge
import geopandas as gpd
from shapely.geometry import box, Polygon
import folium

# Data access
import asf_search as asf
import earthaccess

# Image processing
from scipy.ndimage import uniform_filter, median_filter
from skimage import exposure

# Utilities
import json
from pathlib import Path

print("✓ All libraries imported successfully!")
print(f"ASF Search version: {asf.__version__}")

✓ All libraries imported successfully!
ASF Search version: 10.1.2


In [4]:
#Creating directory structure

directories = {
    'data': './data',
    'data_roi': './data/roi',
    'data_s1': './data/sentinel1',
    'data_s2': './data/sentinel2',
    'output': './output',
    'output_maps': './output/maps',
    'output_changes': './output/changes',
    'reports': './reports'
}

for name, path in directories.items():
    Path(path).mkdir(parents=True, exist_ok=True)
    print(f"✓ Created: {path}")

print("\n" + "="*60)
print("AUTHENTICATION SETUP")
print("="*60)

# Earthdata Authentication
# You'll be prompted to enter credentials
earthaccess.login(strategy="interactive")

print("\n✓ Authentication complete!")
print("✓ Ready to download data from ASF and NASA Earthdata")

✓ Created: ./data
✓ Created: ./data/roi
✓ Created: ./data/sentinel1
✓ Created: ./data/sentinel2
✓ Created: ./output
✓ Created: ./output/maps
✓ Created: ./output/changes
✓ Created: ./reports

AUTHENTICATION SETUP

✓ Authentication complete!
✓ Ready to download data from ASF and NASA Earthdata


In [5]:
# ROI 1: Hirakud Reservoir, Odisha - One of India's longest dams
roi1_hirakud = {
    'name': 'Hirakud_Reservoir',
    'bbox': [83.85, 21.45, 84.05, 21.60],  # [min_lon, min_lat, max_lon, max_lat]
    'center': [83.95, 21.525],
    'description': 'Hirakud Dam reservoir with seasonal water level variations'
}

# ROI 2: Chilika Lake, Odisha - Largest coastal lagoon in India
roi2_chilika = {
    'name': 'Chilika_Lake',
    'bbox': [85.20, 19.65, 85.50, 19.80],
    'center': [85.35, 19.725],
    'description': 'Chilika coastal lagoon with tidal and seasonal variations'
}

rois = [roi1_hirakud, roi2_chilika]

# Define time periods
pre_monsoon = {
    'start': '2024-04-01',
    'end': '2024-05-31',
    'label': 'Pre-Monsoon (Apr-May 2024)'
}

post_monsoon = {
    'start': '2024-09-01',
    'end': '2024-10-31',
    'label': 'Post-Monsoon (Sep-Oct 2024)'
}

# Visualize ROIs
m = folium.Map(location=[20.5, 84.5], zoom_start=7)

for roi in rois:
    # Add rectangle for ROI
    folium.Rectangle(
        bounds=[[roi['bbox'][1], roi['bbox'][0]], [roi['bbox'][3], roi['bbox'][2]]],
        color='red',
        fill=True,
        fillOpacity=0.2,
        popup=f"{roi['name']}<br>{roi['description']}"
    ).add_to(m)

    # Add marker
    folium.Marker(
        location=roi['center'],
        popup=roi['name'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

# Save ROIs as GeoJSON
for roi in rois:
    bbox = roi['bbox']
    geom = box(bbox[0], bbox[1], bbox[2], bbox[3])

    # FIX: convert list fields to strings to avoid GeoJSON error
    roi_clean = roi.copy()
    roi_clean['bbox'] = str(roi_clean['bbox'])
    roi_clean['center'] = str(roi_clean['center'])

    gdf = gpd.GeoDataFrame([roi_clean], geometry=[geom], crs='EPSG:4326')
    gdf.to_file(f"./data/roi/{roi['name']}.geojson", driver='GeoJSON')
    print(f"✓ Saved {roi['name']}.geojson")

# Display map
m.save('./output/maps/roi_locations.html')
print("\n✓ ROI map saved to: ./output/maps/roi_locations.html")
print("✓ Open this file in a browser to view the locations")

# Print summary
print("\n" + "="*60)
print("ROI SUMMARY")
print("="*60)
for roi in rois:
    print(f"\n{roi['name']}:")
    print(f"  Bounding Box: {roi['bbox']}")
    print(f"  Center: {roi['center']}")
    print(f"  {roi['description']}")

print(f"\n{pre_monsoon['label']}: {pre_monsoon['start']} to {pre_monsoon['end']}")
print(f"{post_monsoon['label']}: {post_monsoon['start']} to {post_monsoon['end']}")


✓ Saved Hirakud_Reservoir.geojson
✓ Saved Chilika_Lake.geojson

✓ ROI map saved to: ./output/maps/roi_locations.html
✓ Open this file in a browser to view the locations

ROI SUMMARY

Hirakud_Reservoir:
  Bounding Box: [83.85, 21.45, 84.05, 21.6]
  Center: [83.95, 21.525]
  Hirakud Dam reservoir with seasonal water level variations

Chilika_Lake:
  Bounding Box: [85.2, 19.65, 85.5, 19.8]
  Center: [85.35, 19.725]
  Chilika coastal lagoon with tidal and seasonal variations

Pre-Monsoon (Apr-May 2024): 2024-04-01 to 2024-05-31
Post-Monsoon (Sep-Oct 2024): 2024-09-01 to 2024-10-31


In [6]:
#Search Sentinel-1 GRD data from ASF

print("="*60)
print("SENTINEL-1 DATA SEARCH AND DOWNLOAD")
print("="*60)

def search_sentinel1_asf(roi, start_date, end_date, max_results=5):
  
    bbox = roi['bbox']
    
    
    wkt = f"POLYGON(({bbox[0]} {bbox[1]}, {bbox[2]} {bbox[1]}, {bbox[2]} {bbox[3]}, {bbox[0]} {bbox[3]}, {bbox[0]} {bbox[1]}))"
    
    print(f"\nSearching {roi['name']}: {start_date} to {end_date}")
    
    results = asf.geo_search(
        platform=[asf.PLATFORM.SENTINEL1],
        processingLevel=asf.PRODUCT_TYPE.GRD_HD,  # High resolution GRD
        start=start_date,
        end=end_date,
        intersectsWith=wkt,
        maxResults=max_results
    )
    
    print(f"  Found {len(results)} scenes")
    
    return results

def download_sentinel1_scene(scene, roi_name, period_label, download_dir):
   
    session = asf.ASFSession().auth_with_creds(os.environ.get('EARTHDATA_USERNAME'), 
                                                 os.environ.get('EARTHDATA_PASSWORD'))
    
    
    scene_dir = f"{download_dir}/{roi_name}_{period_label}"
    Path(scene_dir).mkdir(parents=True, exist_ok=True)
    
    print(f"  Downloading: {scene.properties['sceneName']}")
    print(f"  Acquisition: {scene.properties['startTime']}")
    print(f"  Orbit: {scene.properties['flightDirection']}")
    
  
    scene.download(path=scene_dir, session=session)
    
    return scene_dir


s1_scenes = {}


for roi in rois:
    roi_name = roi['name']
    s1_scenes[roi_name] = {}
    
    # Pre-monsoon
    print(f"\n{'='*60}")
    print(f"ROI: {roi_name} - Pre-Monsoon")
    print(f"{'='*60}")
    results_pre = search_sentinel1_asf(roi, pre_monsoon['start'], pre_monsoon['end'], max_results=10)
    
    if len(results_pre) > 0:
        # Filter for best scene (lowest cloud cover, recent date)
        # For GRD, select the most recent scene
        scene_pre = results_pre[0]  # Most recent
        s1_scenes[roi_name]['pre_monsoon'] = scene_pre
        
        print(f"\nSelected scene for Pre-Monsoon:")
        print(f"  Scene: {scene_pre.properties['sceneName']}")
        print(f"  Date: {scene_pre.properties['startTime']}")
        print(f"  Polarization: {scene_pre.properties['polarization']}")
    else:
        print("  WARNING: No scenes found for pre-monsoon period")
    
    # Post-monsoon
    print(f"\n{'='*60}")
    print(f"ROI: {roi_name} - Post-Monsoon")
    print(f"{'='*60}")
    results_post = search_sentinel1_asf(roi, post_monsoon['start'], post_monsoon['end'], max_results=10)
    
    if len(results_post) > 0:
        scene_post = results_post[0]
        s1_scenes[roi_name]['post_monsoon'] = scene_post
        
        print(f"\nSelected scene for Post-Monsoon:")
        print(f"  Scene: {scene_post.properties['sceneName']}")
        print(f"  Date: {scene_post.properties['startTime']}")
        print(f"  Polarization: {scene_post.properties['polarization']}")
    else:
        print("  WARNING: No scenes found for post-monsoon period")

print("\n" + "="*60)
print("SENTINEL-1 SEARCH COMPLETE")
print("="*60)
print(f"Total scenes identified: {sum(len(v) for v in s1_scenes.values())}")

SENTINEL-1 DATA SEARCH AND DOWNLOAD

ROI: Hirakud_Reservoir - Pre-Monsoon

Searching Hirakud_Reservoir: 2024-04-01 to 2024-05-31
  Found 10 scenes

Selected scene for Pre-Monsoon:
  Scene: S1A_IW_GRDH_1SDV_20240523T002148_20240523T002213_053991_069045_4E3F
  Date: 2024-05-23T00:21:48Z
  Polarization: VV+VH

ROI: Hirakud_Reservoir - Post-Monsoon

Searching Hirakud_Reservoir: 2024-09-01 to 2024-10-31
  Found 8 scenes

Selected scene for Post-Monsoon:
  Scene: S1A_IW_GRDH_1SDV_20241026T002147_20241026T002212_056266_06E3CF_B7C4
  Date: 2024-10-26T00:21:47Z
  Polarization: VV+VH

ROI: Chilika_Lake - Pre-Monsoon

Searching Chilika_Lake: 2024-04-01 to 2024-05-31
  Found 10 scenes

Selected scene for Pre-Monsoon:
  Scene: S1A_IW_GRDH_1SDV_20240530T001406_20240530T001431_054093_0693D5_70AF
  Date: 2024-05-30T00:14:06Z
  Polarization: VV+VH

ROI: Chilika_Lake - Post-Monsoon

Searching Chilika_Lake: 2024-09-01 to 2024-10-31
  Found 10 scenes

Selected scene for Post-Monsoon:
  Scene: S1A_IW_GRDH_

In [8]:
#Download Sentinel-1 Data

print("="*60)
print("DOWNLOADING SENTINEL-1 SCENES")
print("="*60)
print("Note: Each scene is approximately 1GB. Download may take 5-10 minutes per scene.")
print("="*60)


import getpass

if 'EARTHDATA_USERNAME' not in os.environ or 'EARTHDATA_PASSWORD' not in os.environ:
    print("\nPlease enter your NASA Earthdata credentials:")
    username = input("Username: ")
    password = getpass.getpass("Password: ")
    os.environ['EARTHDATA_USERNAME'] = username
    os.environ['EARTHDATA_PASSWORD'] = password
else:
    username = os.environ['EARTHDATA_USERNAME']
    password = os.environ['EARTHDATA_PASSWORD']


session = asf.ASFSession().auth_with_creds(username, password)

print("\n✓ Authentication successful!")

downloaded_s1 = {}

for roi_name, periods in s1_scenes.items():
    downloaded_s1[roi_name] = {}
    
    for period_name, scene in periods.items():
        print(f"\n{'='*60}")
        print(f"Downloading: {roi_name} - {period_name}")
        print(f"{'='*60}")
        
        try:
            # Create subdirectory
            scene_dir = f"./data/sentinel1/{roi_name}_{period_name}"
            Path(scene_dir).mkdir(parents=True, exist_ok=True)
            
            print(f"  Scene: {scene.properties['sceneName']}")
            print(f"  Acquisition: {scene.properties['startTime']}")
            print(f"  Polarization: {scene.properties['polarization']}")
            print(f"  Orbit: {scene.properties['flightDirection']}")
            print(f"\n  Starting download...")
            
            
            scene.download(path=scene_dir, session=session)
            
            downloaded_s1[roi_name][period_name] = {
                'path': scene_dir,
                'scene_name': scene.properties['sceneName'],
                'date': scene.properties['startTime']
            }
            print(f"  ✓ Download complete: {scene_dir}")
            
        except Exception as e:
            print(f"  ✗ Download failed: {e}")
            downloaded_s1[roi_name][period_name] = None

print("\n" + "="*60)
print("SENTINEL-1 DOWNLOAD SUMMARY")
print("="*60)
for roi_name, periods in downloaded_s1.items():
    print(f"\n{roi_name}:")
    for period, data in periods.items():
        if data:
            print(f"  {period}: ✓ Success")
            print(f"    Path: {data['path']}")
            print(f"    Scene: {data['scene_name']}")
        else:
            print(f"  {period}: ✗ Failed")


with open('./data/sentinel1/download_log.json', 'w') as f:
    log = {
        'download_time': datetime.now().isoformat(),
        'scenes': downloaded_s1
    }
    json.dump(log, f, indent=2, default=str)

print("\n✓ Download log saved to: ./data/sentinel1/download_log.json")

DOWNLOADING SENTINEL-1 SCENES
Note: Each scene is approximately 1GB. Download may take 5-10 minutes per scene.

✓ Authentication successful!

Downloading: Hirakud_Reservoir - pre_monsoon
  Scene: S1A_IW_GRDH_1SDV_20240523T002148_20240523T002213_053991_069045_4E3F
  Acquisition: 2024-05-23T00:21:48Z
  Polarization: VV+VH
  Orbit: DESCENDING

  Starting download...
  ✓ Download complete: ./data/sentinel1/Hirakud_Reservoir_pre_monsoon

Downloading: Hirakud_Reservoir - post_monsoon
  Scene: S1A_IW_GRDH_1SDV_20241026T002147_20241026T002212_056266_06E3CF_B7C4
  Acquisition: 2024-10-26T00:21:47Z
  Polarization: VV+VH
  Orbit: DESCENDING

  Starting download...
  ✓ Download complete: ./data/sentinel1/Hirakud_Reservoir_post_monsoon

Downloading: Chilika_Lake - pre_monsoon
  Scene: S1A_IW_GRDH_1SDV_20240530T001406_20240530T001431_054093_0693D5_70AF
  Acquisition: 2024-05-30T00:14:06Z
  Polarization: VV+VH
  Orbit: DESCENDING

  Starting download...
  ✓ Download complete: ./data/sentinel1/Chilika

In [9]:
#Search Sentinel-2 L2A from AWS Open Data

print("="*60)
print("SENTINEL-2 DATA SEARCH AND DOWNLOAD")
print("="*60)

# Install additional library if needed
try:
    import pystac_client
    from pystac_client import Client
except:
    !pip install pystac-client --quiet
    import pystac_client
    from pystac_client import Client

import requests
from urllib.parse import urlparse

def search_sentinel2_aws(roi, start_date, end_date, max_cloud_cover=30):
   
    bbox = roi['bbox']
    
    print(f"\nSearching {roi['name']}: {start_date} to {end_date}")
    print(f"  Max cloud cover: {max_cloud_cover}%")
    

    catalog = Client.open("https://earth-search.aws.element84.com/v1")
    

    search = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=bbox,
        datetime=f"{start_date}/{end_date}",
        query={"eo:cloud_cover": {"lt": max_cloud_cover}},
        max_items=20
    )
    
    items = list(search.items())
    print(f"  Found {len(items)} scenes with <{max_cloud_cover}% cloud cover")
    
 
    items_sorted = sorted(items, key=lambda x: x.properties.get('eo:cloud_cover', 100))
    
    return items_sorted

def download_sentinel2_bands(item, roi, bands=['B02', 'B03', 'B04', 'B08', 'B11', 'SCL'], 
                             output_dir='./data/sentinel2'):

    scene_id = item.id
    date = item.properties['datetime'][:10]
    
    print(f"\n  Downloading scene: {scene_id}")
    print(f"  Date: {date}")
    print(f"  Cloud cover: {item.properties.get('eo:cloud_cover', 'N/A')}%")
    
    # Create scene directory
    scene_dir = f"{output_dir}/{roi['name']}_{date}"
    Path(scene_dir).mkdir(parents=True, exist_ok=True)
    
    downloaded_files = {}
    
    for band in bands:
        try:
            if band in item.assets:
                asset = item.assets[band]
                url = asset.href
                
                # Download file
                filename = f"{scene_dir}/{band}.tif"
                
                if not os.path.exists(filename):
                    print(f"    Downloading {band}...", end='')
                    response = requests.get(url, stream=True)
                    response.raise_for_status()
                    
                    with open(filename, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=8192):
                            f.write(chunk)
                    print(" ✓")
                else:
                    print(f"    {band} already exists ✓")
                
                downloaded_files[band] = filename
                
        except Exception as e:
            print(f"    ✗ Failed to download {band}: {e}")
    
    return scene_dir, downloaded_files


s2_items = {}


for roi in rois:
    roi_name = roi['name']
    s2_items[roi_name] = {}
    
   
    print(f"\n{'='*60}")
    print(f"ROI: {roi_name} - Pre-Monsoon")
    print(f"{'='*60}")
    results_pre = search_sentinel2_aws(roi, pre_monsoon['start'], pre_monsoon['end'], max_cloud_cover=30)
    
    if len(results_pre) > 0:
        # Select scene with lowest cloud cover
        scene_pre = results_pre[0]
        s2_items[roi_name]['pre_monsoon'] = scene_pre
        
        print(f"\nSelected scene for Pre-Monsoon:")
        print(f"  Scene ID: {scene_pre.id}")
        print(f"  Date: {scene_pre.properties['datetime'][:10]}")
        print(f"  Cloud cover: {scene_pre.properties.get('eo:cloud_cover', 'N/A')}%")
    else:
        print("  WARNING: No suitable scenes found for pre-monsoon period")
    

    print(f"\n{'='*60}")
    print(f"ROI: {roi_name} - Post-Monsoon")
    print(f"{'='*60}")
    results_post = search_sentinel2_aws(roi, post_monsoon['start'], post_monsoon['end'], max_cloud_cover=30)
    
    if len(results_post) > 0:
        scene_post = results_post[0]
        s2_items[roi_name]['post_monsoon'] = scene_post
        
        print(f"\nSelected scene for Post-Monsoon:")
        print(f"  Scene ID: {scene_post.id}")
        print(f"  Date: {scene_post.properties['datetime'][:10]}")
        print(f"  Cloud cover: {scene_post.properties.get('eo:cloud_cover', 'N/A')}%")
    else:
        print("  WARNING: No suitable scenes found for post-monsoon period")

print("\n" + "="*60)
print("SENTINEL-2 SEARCH COMPLETE")
print("="*60)
print(f"Total scenes identified: {sum(len(v) for v in s2_items.values())}")

SENTINEL-2 DATA SEARCH AND DOWNLOAD

ROI: Hirakud_Reservoir - Pre-Monsoon

Searching Hirakud_Reservoir: 2024-04-01 to 2024-05-31
  Max cloud cover: 30%
  Found 20 scenes with <30% cloud cover

Selected scene for Pre-Monsoon:
  Scene ID: S2A_44QQJ_20240514_0_L2A
  Date: 2024-05-14
  Cloud cover: 0.022878%

ROI: Hirakud_Reservoir - Post-Monsoon

Searching Hirakud_Reservoir: 2024-09-01 to 2024-10-31
  Max cloud cover: 30%
  Found 18 scenes with <30% cloud cover

Selected scene for Post-Monsoon:
  Scene ID: S2B_44QQK_20241006_0_L2A
  Date: 2024-10-06
  Cloud cover: 10.98659%

ROI: Chilika_Lake - Pre-Monsoon

Searching Chilika_Lake: 2024-04-01 to 2024-05-31
  Max cloud cover: 30%
  Found 15 scenes with <30% cloud cover

Selected scene for Pre-Monsoon:
  Scene ID: S2B_45QUC_20240416_0_L2A
  Date: 2024-04-16
  Cloud cover: 0.03637%

ROI: Chilika_Lake - Post-Monsoon

Searching Chilika_Lake: 2024-09-01 to 2024-10-31
  Max cloud cover: 30%
  Found 7 scenes with <30% cloud cover

Selected scene f

In [10]:
# Download Sentinel-2 data

print("="*60)
print("CHECKING AVAILABLE SENTINEL-2 ASSETS")
print("="*60)

# First, let's see what assets are actually available
for roi_name, periods in s2_items.items():
    for period_name, item in periods.items():
        print(f"\n{roi_name} - {period_name}:")
        print(f"  Scene: {item.id}")
        print(f"  Available assets: {list(item.assets.keys())[:10]}...")  # Show first 10
        break
    break

print("\n" + "="*60)
print("DOWNLOADING SENTINEL-2 BANDS")
print("="*60)


BAND_MAPPING = {
    'B02': ['B02', 'blue', 'coastal'],
    'B03': ['B03', 'green'],
    'B04': ['B04', 'red'],
    'B08': ['B08', 'nir', 'nir08'],
    'B11': ['B11', 'swir16', 'swir1'],
    'SCL': ['SCL', 'scl']
}

def find_band_asset(item, band_key):
    """Find the actual asset name for a band"""
    possible_names = BAND_MAPPING.get(band_key, [band_key])
    
    for name in possible_names:
        if name in item.assets:
            return name
        # Try lowercase
        if name.lower() in item.assets:
            return name.lower()
    
    return None

def download_sentinel2_bands_v2(item, roi_name, period_name, target_bands=['B02', 'B03', 'B04', 'B08', 'B11', 'SCL']):

    scene_id = item.id
    date = item.properties['datetime'][:10]
    cloud_cover = item.properties.get('eo:cloud_cover', 'N/A')
    
    print(f"\n  Scene: {scene_id}")
    print(f"  Date: {date}")
    print(f"  Cloud cover: {cloud_cover}%")
    
    # Create scene directory
    scene_dir = f"./data/sentinel2/{roi_name}_{date}"
    Path(scene_dir).mkdir(parents=True, exist_ok=True)
    
    downloaded_files = {}
    
    for band_key in target_bands:
        try:
   
            asset_name = find_band_asset(item, band_key)
            
            if asset_name:
                asset = item.assets[asset_name]
                url = asset.href
                
   
                filename = f"{scene_dir}/{band_key}.tif"
                
                if not os.path.exists(filename):
                    print(f"    Downloading {band_key} (as {asset_name})...", end='', flush=True)
                    
     
                    response = requests.get(url, stream=True, timeout=300)
                    response.raise_for_status()
                    
                    with open(filename, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=1024*1024):  # 1MB chunks
                            if chunk:
                                f.write(chunk)
                    
               
                    if os.path.exists(filename) and os.path.getsize(filename) > 0:
                        file_size_mb = os.path.getsize(filename) / (1024*1024)
                        print(f" ✓ ({file_size_mb:.1f} MB)")
                        downloaded_files[band_key] = filename
                    else:
                        print(f" ✗ (file empty)")
                        
                else:
                    file_size_mb = os.path.getsize(filename) / (1024*1024)
                    print(f"    {band_key} already exists ✓ ({file_size_mb:.1f} MB)")
                    downloaded_files[band_key] = filename
            else:
                print(f"    {band_key} not found in available assets")
                
        except Exception as e:
            print(f"    ✗ Failed to download {band_key}: {str(e)[:80]}")
    
    return scene_dir, downloaded_files

downloaded_s2 = {}

for roi_name, periods in s2_items.items():
    downloaded_s2[roi_name] = {}
    
    for period_name, item in periods.items():
        print(f"\n{'='*60}")
        print(f"Downloading: {roi_name} - {period_name}")
        print(f"{'='*60}")
        
        try:
            scene_dir, files = download_sentinel2_bands_v2(
                item,
                roi_name,
                period_name,
                target_bands=['B02', 'B03', 'B04', 'B08', 'B11', 'SCL']
            )
            
            downloaded_s2[roi_name][period_name] = {
                'path': scene_dir,
                'scene_id': item.id,
                'date': item.properties['datetime'][:10],
                'cloud_cover': item.properties.get('eo:cloud_cover', 'N/A'),
                'files': files
            }
            
            if len(files) > 0:
                print(f"\n  ✓ Successfully downloaded {len(files)} bands to: {scene_dir}")
            else:
                print(f"\n  ✗ No bands were downloaded")
            
        except Exception as e:
            print(f"\n  ✗ Download failed: {e}")
            downloaded_s2[roi_name][period_name] = None

print("\n" + "="*60)
print("SENTINEL-2 DOWNLOAD SUMMARY")
print("="*60)

total_bands = 0
for roi_name, periods in downloaded_s2.items():
    print(f"\n{roi_name}:")
    for period, data in periods.items():
        if data and data['files']:
            print(f"  {period}: ✓ Success")
            print(f"    Path: {data['path']}")
            print(f"    Scene: {data['scene_id']}")
            print(f"    Cloud cover: {data['cloud_cover']}%")
            print(f"    Bands: {', '.join(data['files'].keys())}")
            total_bands += len(data['files'])
        else:
            print(f"  {period}: ✗ Failed or no files downloaded")

print(f"\nTotal bands downloaded: {total_bands}")


with open('./data/sentinel2/download_log.json', 'w') as f:
    log = {
        'download_time': datetime.now().isoformat(),
        'scenes': downloaded_s2
    }
    json.dump(log, f, indent=2, default=str)

print("\n✓ Download log saved to: ./data/sentinel2/download_log.json")

CHECKING AVAILABLE SENTINEL-2 ASSETS

Hirakud_Reservoir - pre_monsoon:
  Scene: S2A_44QQJ_20240514_0_L2A
  Available assets: ['aot', 'blue', 'coastal', 'granule_metadata', 'green', 'nir', 'nir08', 'nir09', 'red', 'rededge1']...

DOWNLOADING SENTINEL-2 BANDS

Downloading: Hirakud_Reservoir - pre_monsoon

  Scene: S2A_44QQJ_20240514_0_L2A
  Date: 2024-05-14
  Cloud cover: 0.022878%
 ✓ (214.9 MB)ng B02 (as blue)...
 ✓ (217.5 MB)ng B03 (as green)...
 ✓ (227.8 MB)ng B04 (as red)...
 ✓ (232.2 MB)ng B08 (as nir)...
 ✓ (58.2 MB)ing B11 (as swir16)...
 ✓ (3.7 MB)ding SCL (as scl)...

  ✓ Successfully downloaded 6 bands to: ./data/sentinel2/Hirakud_Reservoir_2024-05-14

Downloading: Hirakud_Reservoir - post_monsoon

  Scene: S2B_44QQK_20241006_0_L2A
  Date: 2024-10-06
  Cloud cover: 10.98659%
 ✓ (198.0 MB)ng B02 (as blue)...
 ✓ (206.6 MB)ng B03 (as green)...
 ✓ (206.2 MB)ng B04 (as red)...
 ✓ (230.5 MB)ng B08 (as nir)...
 ✓ (56.7 MB)ing B11 (as swir16)...
 ✓ (2.3 MB)ding SCL (as scl)...

  ✓ Suc

In [24]:
#Sentinel-1 Preprocessing

print("="*60)
print("SENTINEL-1 PREPROCESSING - HANDLE VV AND VH")
print("="*60)

import zipfile
import glob

def extract_sentinel1_zip(zip_path, extract_dir):
    """Extract Sentinel-1 zip file"""
    print(f"    Extracting: {os.path.basename(zip_path)}")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    return extract_dir

def find_sentinel1_sar_band(scene_dir):
    """Find VV or VH band (prefer VV)"""
    # Try VV first
    vv_patterns = [
        f"{scene_dir}/**/*-vv-*.tiff",
        f"{scene_dir}/**/*-vv-*.tif",
    ]
    
    for pattern in vv_patterns:
        files = glob.glob(pattern, recursive=True)
        if files:
            return files[0], 'VV'
    
    # Fall back to VH
    vh_patterns = [
        f"{scene_dir}/**/*-vh-*.tiff",
        f"{scene_dir}/**/*-vh-*.tif",
    ]
    
    for pattern in vh_patterns:
        files = glob.glob(pattern, recursive=True)
        if files:
            return files[0], 'VH'
    
    return None, None

def lee_filter(img, window_size=5):
    """Apply Lee speckle filter"""
    img = np.where(np.isfinite(img), img, -30)
    mean = uniform_filter(img, size=window_size)
    sqr_mean = uniform_filter(img**2, size=window_size)
    variance = sqr_mean - mean**2
    variance = np.where(variance < 0, 0, variance)
    overall_variance = np.var(img[np.isfinite(img)])
    weights = variance / (variance + overall_variance + 1e-10)
    filtered = mean + weights * (img - mean)
    return filtered

def process_sentinel1_scene_flexible(scene_path, roi, output_dir):
    """
    Process Sentinel-1 with flexible polarization
    """
    print(f"\n  Processing: {os.path.basename(scene_path)}")
    

    zip_files = glob.glob(f"{scene_path}/*.zip")
    if not zip_files:
        print(f"    ✗ No zip files found")
        return None
    
    zip_path = zip_files[0]
    safe_dir = zip_path.replace('.zip', '.SAFE')
    
    if not os.path.exists(safe_dir):
        print(f"    Extracting...")
        extract_sentinel1_zip(zip_path, os.path.dirname(zip_path))
    else:
        print(f"    Already extracted")
    

    sar_path, polarization = find_sentinel1_sar_band(safe_dir)
    
    if not sar_path:
        print(f"    ✗ Could not find any SAR band")
        return None
    
    print(f"    Found {polarization}: {os.path.basename(sar_path)}")
    

    with rasterio.open(sar_path) as src:
        sar_linear = src.read(1)
        original_shape = sar_linear.shape
        print(f"    Original shape: {original_shape}")
    

    h, w = sar_linear.shape
    

    h_start = int(h * 0.2)
    h_end = int(h * 0.8)
    w_start = int(w * 0.2)
    w_end = int(w * 0.8)
    
    sar_subset = sar_linear[h_start:h_end, w_start:w_end]
    print(f"    Subset shape: {sar_subset.shape}")
    

    sar_subset = np.where(sar_subset > 0, sar_subset, 0.0001)
    sar_db = 10 * np.log10(sar_subset)
    
    print(f"    {polarization} range: {sar_db.min():.2f} to {sar_db.max():.2f} dB")
    

    print(f"    Applying Lee filter...")
    sar_filtered = lee_filter(sar_db, window_size=5)
    

    output_path = f"{output_dir}/S1_{polarization}_processed.tif"
    

    bbox = roi['bbox']
    
    from rasterio.transform import from_bounds
    transform = from_bounds(bbox[0], bbox[1], bbox[2], bbox[3], 
                           sar_filtered.shape[1], sar_filtered.shape[0])
    
    profile = {
        'driver': 'GTiff',
        'dtype': rasterio.float32,
        'width': sar_filtered.shape[1],
        'height': sar_filtered.shape[0],
        'count': 1,
        'crs': 'EPSG:4326',
        'transform': transform,
        'compress': 'lzw'
    }
    
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(sar_filtered.astype(rasterio.float32), 1)
    
    print(f"    ✓ Saved: {output_path}")
    print(f"    Output shape: {sar_filtered.shape}")
    
    return output_path, polarization


processed_s1 = {}

for roi in rois:
    roi_name = roi['name']
    processed_s1[roi_name] = {}
    
    print(f"\n{'='*60}")
    print(f"Processing: {roi_name}")
    print(f"{'='*60}")
    
    for period in ['pre_monsoon', 'post_monsoon']:
        if roi_name in downloaded_s1 and period in downloaded_s1[roi_name]:
            scene_data = downloaded_s1[roi_name][period]
            scene_path = scene_data.get('path') if isinstance(scene_data, dict) else scene_data
            
            if scene_path:
                output_dir = f"./output/{roi_name}/s1_{period}"
                Path(output_dir).mkdir(parents=True, exist_ok=True)
                
                try:
                    result = process_sentinel1_scene_flexible(scene_path, roi, output_dir)
                    if result:
                        processed_path, pol = result
                        processed_s1[roi_name][period] = {
                            'path': processed_path,
                            'polarization': pol
                        }
                    else:
                        processed_s1[roi_name][period] = None
                except Exception as e:
                    print(f"    ✗ Failed: {e}")
                    import traceback
                    traceback.print_exc()
                    processed_s1[roi_name][period] = None

print("\n" + "="*60)
print("SENTINEL-1 PREPROCESSING COMPLETE")
print("="*60)

success_count = 0
for roi_name, periods in processed_s1.items():
    print(f"\n{roi_name}:")
    for period, data in periods.items():
        if data:
            if isinstance(data, dict):
                print(f"  {period}: ✓ {data['polarization']} - {data['path']}")
            else:
                print(f"  {period}: ✓ {data}")
            success_count += 1
        else:
            print(f"  {period}: ✗ Failed")

print(f"\nSuccessfully processed: {success_count}/4 scenes")

# Save log
with open('./output/s1_preprocessing_log.json', 'w') as f:
    json.dump(processed_s1, f, indent=2, default=str)
print("✓ Log saved")

SENTINEL-1 PREPROCESSING - HANDLE VV AND VH

Processing: Hirakud_Reservoir

  Processing: Hirakud_Reservoir_pre_monsoon
    Already extracted
    Found VV: s1a-iw-grd-vv-20240523t002148-20240523t002213-053991-069045-001.tiff
    Original shape: (16740, 25521)
    Subset shape: (10044, 15312)
    VV range: 9.54 to 39.99 dB
    Applying Lee filter...
    ✓ Saved: ./output/Hirakud_Reservoir/s1_pre_monsoon/S1_VV_processed.tif
    Output shape: (10044, 15312)

  Processing: Hirakud_Reservoir_post_monsoon
    Already extracted
    Found VV: s1a-iw-grd-vv-20241026t002147-20241026t002212-056266-06e3cf-001.tiff
    Original shape: (16740, 25523)
    Subset shape: (10044, 15314)
    VV range: 10.00 to 39.91 dB
    Applying Lee filter...
    ✓ Saved: ./output/Hirakud_Reservoir/s1_post_monsoon/S1_VV_processed.tif
    Output shape: (10044, 15314)

Processing: Chilika_Lake

  Processing: Chilika_Lake_pre_monsoon
    Already extracted
    Found VV: s1a-iw-grd-vv-20240530t001406-20240530t001431-054093

In [ ]:
#Preprocess Sentinel-2

print("="*60)
print("SENTINEL-2 PREPROCESSING")
print("="*60)

from scipy.ndimage import zoom

def calculate_ndwi(green_path, nir_path):
    """
    Calculate NDWI (Normalized Difference Water Index)
    NDWI = (Green - NIR) / (Green + NIR)
    """
    with rasterio.open(green_path) as green_src:
        green = green_src.read(1).astype(float)
        profile = green_src.profile.copy()
    
    with rasterio.open(nir_path) as nir_src:
        nir = nir_src.read(1).astype(float)
    

    denominator = green + nir
    ndwi = np.where(denominator != 0, (green - nir) / denominator, 0)
    
    return ndwi, profile

def apply_cloud_mask(data, scl_path):
    """
    Apply cloud mask using Scene Classification Layer (SCL)
    Handles resolution mismatch by resampling SCL
    """
    with rasterio.open(scl_path) as scl_src:
        scl = scl_src.read(1)
    

    if scl.shape != data.shape:
        print(f"      Resampling SCL from {scl.shape} to {data.shape}")
        
        zoom_factor_h = data.shape[0] / scl.shape[0]
        zoom_factor_w = data.shape[1] / scl.shape[1]
        
        scl = zoom(scl, (zoom_factor_h, zoom_factor_w), order=0)
    

    cloud_mask = np.isin(scl, [3, 8, 9, 10])
    

    data_masked = data.copy()
    data_masked[cloud_mask] = np.nan
    
    cloud_percentage = (np.sum(cloud_mask) / cloud_mask.size) * 100
    
    return data_masked, cloud_percentage

def create_rgb_composite(scene_dir, output_path):
 
    b04_path = f"{scene_dir}/B04.tif"  # Red
    b03_path = f"{scene_dir}/B03.tif"  # Green
    b02_path = f"{scene_dir}/B02.tif"  # Blue
    
    with rasterio.open(b04_path) as src:
        red = src.read(1)
        profile = src.profile.copy()
    
    with rasterio.open(b03_path) as src:
        green = src.read(1)
    
    with rasterio.open(b02_path) as src:
        blue = src.read(1)
    
    # Normalize to 0-1 for visualization
    def normalize(band):
        band = band.astype(float)
        valid_pixels = band[band > 0]
        if len(valid_pixels) > 0:
            p2, p98 = np.percentile(valid_pixels, (2, 98))
            return np.clip((band - p2) / (p98 - p2), 0, 1)
        else:
            return band
    
    red_norm = normalize(red)
    green_norm = normalize(green)
    blue_norm = normalize(blue)
    
    # Stack and save
    rgb = np.stack([red_norm, green_norm, blue_norm])
    
    profile.update(count=3, dtype=rasterio.float32)
    
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(rgb.astype(rasterio.float32))
    
    return output_path

def process_sentinel2_scene(scene_data, roi_name, period, output_dir):
 
    scene_dir = scene_data['path']
    scene_id = scene_data['scene_id']
    cloud_cover = scene_data['cloud_cover']
    
    print(f"\n  Processing: {scene_id}")
    print(f"  Reported cloud cover: {cloud_cover}%")
    
    green_path = f"{scene_dir}/B03.tif"
    nir_path = f"{scene_dir}/B08.tif"
    scl_path = f"{scene_dir}/SCL.tif"
    

    if not os.path.exists(green_path):
        print(f"    ✗ Band files not found in {scene_dir}")
        return None
    

    print(f"    Calculating NDWI...")
    ndwi, profile = calculate_ndwi(green_path, nir_path)
    print(f"    NDWI range: {np.nanmin(ndwi):.3f} to {np.nanmax(ndwi):.3f}")
    print(f"    NDWI shape: {ndwi.shape}")
    

    if os.path.exists(scl_path):
        print(f"    Applying cloud mask...")
        ndwi_masked, actual_cloud_pct = apply_cloud_mask(ndwi, scl_path)
        print(f"    Actual cloud coverage: {actual_cloud_pct:.2f}%")
    else:
        print(f"    Warning: No SCL file, skipping cloud masking")
        ndwi_masked = ndwi
        actual_cloud_pct = 0
    

    ndwi_path = f"{output_dir}/S2_NDWI.tif"
    profile.update(dtype=rasterio.float32, count=1, nodata=np.nan)
    
    with rasterio.open(ndwi_path, 'w', **profile) as dst:
        dst.write(ndwi_masked.astype(rasterio.float32), 1)
    
    print(f"    ✓ Saved NDWI: {ndwi_path}")
    

    print(f"    Creating RGB composite...")
    rgb_path = f"{output_dir}/S2_RGB.tif"
    create_rgb_composite(scene_dir, rgb_path)
    print(f"    ✓ Saved RGB: {rgb_path}")
    
    return {
        'ndwi_path': ndwi_path,
        'rgb_path': rgb_path,
        'actual_cloud_pct': actual_cloud_pct,
        'shape': ndwi_masked.shape
    }


processed_s2 = {}

for roi_name, periods in downloaded_s2.items():
    processed_s2[roi_name] = {}
    
    print(f"\n{'='*60}")
    print(f"Processing: {roi_name}")
    print(f"{'='*60}")
    
    for period_name, scene_data in periods.items():
        if scene_data and scene_data.get('path'):
            output_dir = f"./output/{roi_name}/s2_{period_name}"
            Path(output_dir).mkdir(parents=True, exist_ok=True)
            
            try:
                result = process_sentinel2_scene(scene_data, roi_name, period_name, output_dir)
                processed_s2[roi_name][period_name] = result
            except Exception as e:
                print(f"    ✗ Processing failed: {e}")
                import traceback
                traceback.print_exc()
                processed_s2[roi_name][period_name] = None
        else:
            print(f"\n  {period_name}: No data available")
            processed_s2[roi_name][period_name] = None

print("\n" + "="*60)
print("SENTINEL-2 PREPROCESSING COMPLETE")
print("="*60)

success_count = 0
for roi_name, periods in processed_s2.items():
    print(f"\n{roi_name}:")
    for period, result in periods.items():
        if result:
            print(f"  {period}: ✓")
            print(f"    NDWI: {result['ndwi_path']}")
            print(f"    RGB: {result['rgb_path']}")
            print(f"    Cloud: {result['actual_cloud_pct']:.1f}%")
            print(f"    Shape: {result['shape']}")
            success_count += 1
        else:
            print(f"  {period}: ✗ Failed")

print(f"\nSuccessfully processed: {success_count}/4 scenes")


with open('./output/s2_preprocessing_log.json', 'w') as f:
    json.dump(processed_s2, f, indent=2, default=str)
print("✓ Log saved")

SENTINEL-2 PREPROCESSING

Processing: Hirakud_Reservoir

  Processing: S2A_44QQJ_20240514_0_L2A
  Reported cloud cover: 0.022878%
    Calculating NDWI...
    NDWI range: -1.000 to 1.000
    NDWI shape: (10980, 10980)
    Applying cloud mask...
      Resampling SCL from (5490, 5490) to (10980, 10980)
    Actual cloud coverage: 0.03%
    ✓ Saved NDWI: ./output/Hirakud_Reservoir/s2_pre_monsoon/S2_NDWI.tif
    Creating RGB composite...
    ✓ Saved RGB: ./output/Hirakud_Reservoir/s2_pre_monsoon/S2_RGB.tif

  Processing: S2B_44QQK_20241006_0_L2A
  Reported cloud cover: 10.98659%
    Calculating NDWI...
    NDWI range: -1.000 to 1.000
    NDWI shape: (10980, 10980)
    Applying cloud mask...
      Resampling SCL from (5490, 5490) to (10980, 10980)
    Actual cloud coverage: 14.63%
    ✓ Saved NDWI: ./output/Hirakud_Reservoir/s2_post_monsoon/S2_NDWI.tif
    Creating RGB composite...
    ✓ Saved RGB: ./output/Hirakud_Reservoir/s2_post_monsoon/S2_RGB.tif

Processing: Chilika_Lake

  Processing: 

In [13]:
#Coregister

print("="*60)
print("COREGISTRATION: ALIGN S1 AND S2")
print("="*60)

from rasterio.warp import calculate_default_transform, reproject, Resampling

def coregister_to_reference(src_path, ref_path, output_path):
   
    with rasterio.open(ref_path) as ref:
        ref_crs = ref.crs
        ref_transform = ref.transform
        ref_width = ref.width
        ref_height = ref.height
    
 
    with rasterio.open(src_path) as src:

        src_data = src.read(1)
        
 
        dst_data = np.empty((ref_height, ref_width), dtype=src_data.dtype)
        

        reproject(
            source=src_data,
            destination=dst_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=ref_transform,
            dst_crs=ref_crs,
            resampling=Resampling.bilinear
        )
        

        profile = src.profile.copy()
        profile.update({
            'crs': ref_crs,
            'transform': ref_transform,
            'width': ref_width,
            'height': ref_height
        })
        
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(dst_data, 1)
    
    return output_path


coregistered = {}


roi_names = ['Hirakud_Reservoir', 'Chilika_Lake']

for roi_name in roi_names:
    coregistered[roi_name] = {}
    
    print(f"\n{'='*60}")
    print(f"Coregistering: {roi_name}")
    print(f"{'='*60}")
    
    for period in ['pre_monsoon', 'post_monsoon']:
        print(f"\n  Period: {period}")
        

        s1_data = processed_s1.get(roi_name, {}).get(period)
        s2_data = processed_s2.get(roi_name, {}).get(period)
        
        if not s1_data or not s2_data:
            print(f"    ✗ Missing data for {period}")
            continue
        

        if isinstance(s1_data, dict):
            s1_path = s1_data['path']
        else:
            s1_path = s1_data
        
 
        s2_ndwi_path = s2_data['ndwi_path']
        
        print(f"    Reference (S2): {os.path.basename(s2_ndwi_path)}")
        print(f"    Source (S1): {os.path.basename(s1_path)}")
        

        output_dir = f"./output/{roi_name}/coregistered_{period}"
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        

        s1_coreg_path = f"{output_dir}/S1_coregistered.tif"
        
        try:
            print(f"    Coregistering S1 to S2 grid...")
            coregister_to_reference(s1_path, s2_ndwi_path, s1_coreg_path)
            
 
            with rasterio.open(s1_coreg_path) as s1_src:
                s1_shape = (s1_src.height, s1_src.width)
            
            with rasterio.open(s2_ndwi_path) as s2_src:
                s2_shape = (s2_src.height, s2_src.width)
            
            if s1_shape == s2_shape:
                print(f"    ✓ Coregistration successful")
                print(f"      Shape: {s1_shape}")
                
                coregistered[roi_name][period] = {
                    's1_coreg': s1_coreg_path,
                    's2_ndwi': s2_ndwi_path,
                    's2_rgb': s2_data['rgb_path'],
                    'shape': s1_shape
                }
            else:
                print(f"    ✗ Shape mismatch!")
                print(f"      S1: {s1_shape}, S2: {s2_shape}")
                coregistered[roi_name][period] = None
                
        except Exception as e:
            print(f"    ✗ Coregistration failed: {e}")
            import traceback
            traceback.print_exc()
            coregistered[roi_name][period] = None

print("\n" + "="*60)
print("COREGISTRATION COMPLETE")
print("="*60)

success_count = 0
for roi_name, periods in coregistered.items():
    print(f"\n{roi_name}:")
    for period, data in periods.items():
        if data:
            print(f"  {period}: ✓ Shape {data['shape']}")
            success_count += 1
        else:
            print(f"  {period}: ✗ Failed")

print(f"\nSuccessfully coregistered: {success_count}/4 scene pairs")

# Save log
with open('./output/coregistration_log.json', 'w') as f:
    json.dump(coregistered, f, indent=2, default=str)
print("✓ Log saved")

COREGISTRATION: ALIGN S1 AND S2

Coregistering: Hirakud_Reservoir

  Period: pre_monsoon
    Reference (S2): S2_NDWI.tif
    Source (S1): S1_VV_processed.tif
    Coregistering S1 to S2 grid...
    ✓ Coregistration successful
      Shape: (10980, 10980)

  Period: post_monsoon
    Reference (S2): S2_NDWI.tif
    Source (S1): S1_VV_processed.tif
    Coregistering S1 to S2 grid...
    ✓ Coregistration successful
      Shape: (10980, 10980)

Coregistering: Chilika_Lake

  Period: pre_monsoon
    Reference (S2): S2_NDWI.tif
    Source (S1): S1_VV_processed.tif
    Coregistering S1 to S2 grid...
    ✓ Coregistration successful
      Shape: (10980, 10980)

  Period: post_monsoon
    Reference (S2): S2_NDWI.tif
    Source (S1): S1_VV_processed.tif
    Coregistering S1 to S2 grid...
    ✓ Coregistration successful
      Shape: (10980, 10980)

COREGISTRATION COMPLETE

Hirakud_Reservoir:
  pre_monsoon: ✓ Shape (10980, 10980)
  post_monsoon: ✓ Shape (10980, 10980)

Chilika_Lake:
  pre_monsoon: ✓ S

In [3]:
# CHUNK 13: Change Detection Analysis

print("="*60)
print("CHANGE DETECTION ANALYSIS")
print("="*60)

# Import necessary libraries
import numpy as np
import rasterio
import json
from pathlib import Path

# Check if prerequisites exist
if 'water_masks' not in globals():
    print("\n⚠️  ERROR: water_masks not found!")
    print("You need to run CHUNK 12 first to generate water masks.")
    print("\nAttempting to load from saved files...")
    
    # Try to reconstruct from saved files
    if Path('./output/water_masks_log.json').exists():
        print("✓ Found water_masks_log.json, loading...")
        with open('./output/water_masks_log.json', 'r') as f:
            water_masks_meta = json.load(f)
        
        # Reconstruct water_masks with actual arrays
        water_masks = {}
        for roi_name in ['Hirakud_Reservoir', 'Chilika_Lake']:
            water_masks[roi_name] = {}
            for period in ['pre_monsoon', 'post_monsoon']:
                if roi_name in water_masks_meta and period in water_masks_meta[roi_name]:
                    paths = water_masks_meta[roi_name][period]
                    
                    # Load the masks from files
                    with rasterio.open(paths['s2_water']) as src:
                        s2_mask = src.read(1)
                    
                    with rasterio.open(paths['combined_water']) as src:
                        combined_mask = src.read(1)
                    
                    water_masks[roi_name][period] = {
                        's2_mask': s2_mask,
                        'combined_mask': combined_mask,
                        's2_water': paths['s2_water'],
                        'combined_water': paths['combined_water']
                    }
        print("✓ Water masks reconstructed from saved files")
    else:
        print("\n❌ Cannot proceed without water masks.")
        print("Please run CHUNK 12 first, then run this cell again.")
        raise SystemExit("Missing prerequisites")

def compute_change(pre_mask, post_mask):
    """
    Compute change between pre and post monsoon
    Returns:
    - water_gain: areas that became water
    - water_loss: areas that lost water
    - no_change_water: persistent water
    - no_change_land: persistent land
    """
    water_gain = (pre_mask == 0) & (post_mask == 1)
    water_loss = (pre_mask == 1) & (post_mask == 0)
    no_change_water = (pre_mask == 1) & (post_mask == 1)
    no_change_land = (pre_mask == 0) & (post_mask == 0)
    
    # Create change map: 0=no change land, 1=persistent water, 2=water gain, 3=water loss
    change_map = np.zeros_like(pre_mask, dtype=np.uint8)
    change_map[no_change_water] = 1
    change_map[water_gain] = 2
    change_map[water_loss] = 3
    
    # Calculate statistics
    total_pixels = pre_mask.size
    stats = {
        'water_gain_pixels': int(np.sum(water_gain)),
        'water_loss_pixels': int(np.sum(water_loss)),
        'persistent_water_pixels': int(np.sum(no_change_water)),
        'persistent_land_pixels': int(np.sum(no_change_land)),
        'water_gain_pct': (np.sum(water_gain) / total_pixels) * 100,
        'water_loss_pct': (np.sum(water_loss) / total_pixels) * 100,
        'net_change_pct': ((np.sum(water_gain) - np.sum(water_loss)) / total_pixels) * 100
    }
    
    return change_map, stats

# Perform change detection for each ROI
change_results = {}

for roi_name in water_masks.keys():
    print(f"\n{'='*60}")
    print(f"Change Detection: {roi_name}")
    print(f"{'='*60}")
    
    # Get pre and post masks
    pre_data = water_masks[roi_name].get('pre_monsoon')
    post_data = water_masks[roi_name].get('post_monsoon')
    
    if not pre_data or not post_data:
        print(f"  ✗ Missing data")
        continue
    
    output_dir = f"./output/{roi_name}/change_detection"
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Analyze each sensor combination
    change_results[roi_name] = {}
    
    for sensor_type in ['s2', 'combined']:
        mask_key = f'{sensor_type}_mask'
        print(f"\n  Analyzing {sensor_type.upper()} change:")
        
        pre_mask = pre_data[mask_key]
        post_mask = post_data[mask_key]
        
        # Compute change
        change_map, stats = compute_change(pre_mask, post_mask)
        
        # Print statistics
        print(f"    Water gain: {stats['water_gain_pct']:.2f}% ({stats['water_gain_pixels']:,} pixels)")
        print(f"    Water loss: {stats['water_loss_pct']:.2f}% ({stats['water_loss_pixels']:,} pixels)")
        print(f"    Net change: {stats['net_change_pct']:+.2f}%")
        print(f"    Persistent water: {stats['persistent_water_pixels']:,} pixels")
        
        # Save change map
        change_path = f"{output_dir}/{sensor_type}_change_map.tif"
        
        # Get profile from one of the water masks
        with rasterio.open(pre_data[f'{sensor_type}_water']) as src:
            profile = src.profile.copy()
        
        profile.update(dtype=rasterio.uint8, count=1, nodata=255)
        
        with rasterio.open(change_path, 'w', **profile) as dst:
            dst.write(change_map, 1)
        
        print(f"    ✓ Saved: {change_path}")
        
        change_results[roi_name][sensor_type] = {
            'change_map_path': change_path,
            'statistics': stats,
            'change_map_array': change_map
        }

print("\n" + "="*60)
print("CHANGE DETECTION SUMMARY")
print("="*60)

for roi_name, sensors in change_results.items():
    print(f"\n{roi_name}:")
    for sensor_type, data in sensors.items():
        stats = data['statistics']
        print(f"  {sensor_type.upper()}:")
        print(f"    Net water change: {stats['net_change_pct']:+.2f}%")
        print(f"    Water gain: {stats['water_gain_pct']:.2f}%")
        print(f"    Water loss: {stats['water_loss_pct']:.2f}%")

# Save statistics to JSON
stats_log = {}
for roi_name, sensors in change_results.items():
    stats_log[roi_name] = {}
    for sensor_type, data in sensors.items():
        stats_log[roi_name][sensor_type] = {
            'change_map': data['change_map_path'],
            'statistics': data['statistics']
        }

with open('./output/change_detection_stats.json', 'w') as f:
    json.dump(stats_log, f, indent=2)

print("\n✓ Statistics saved to: ./output/change_detection_stats.json")
print("✓ Change detection complete!")

CHANGE DETECTION ANALYSIS

Change Detection: Hirakud_Reservoir

  Analyzing S2 change:
    Water gain: 4.68% (5,645,948 pixels)
    Water loss: 1.81% (2,178,565 pixels)
    Net change: +2.88%
    Persistent water: 44,235 pixels
    ✓ Saved: ./output/Hirakud_Reservoir/change_detection/s2_change_map.tif

  Analyzing COMBINED change:
    Water gain: 4.66% (5,623,483 pixels)
    Water loss: 3.40% (4,102,789 pixels)
    Net change: +1.26%
    Persistent water: 78,056 pixels
    ✓ Saved: ./output/Hirakud_Reservoir/change_detection/combined_change_map.tif

Change Detection: Chilika_Lake

  Analyzing S2 change:
    Water gain: 72.69% (87,639,160 pixels)
    Water loss: 0.38% (456,485 pixels)
    Net change: +72.31%
    Persistent water: 2,482,314 pixels
    ✓ Saved: ./output/Chilika_Lake/change_detection/s2_change_map.tif

  Analyzing COMBINED change:
    Water gain: 74.10% (89,340,787 pixels)
    Water loss: 0.37% (445,077 pixels)
    Net change: +73.74%
    Persistent water: 2,499,752 pixels

In [5]:
# Generate Water Masks

print("="*60)
print("GENERATING WATER MASKS (S2 & COMBINED)")
print("="*60)


import numpy as np
import rasterio
import json
from pathlib import Path


if 'coregistered' not in globals():
    print("\n⚠️  ERROR: coregistered not found!")
    print("You need to run the coregistration chunk first.")
    print("\nAttempting to load from saved files...")
    

    if Path('./output/coregistration_log.json').exists():
        print("✓ Found coregistration_log.json, loading...")
        with open('./output/coregistration_log.json', 'r') as f:
            coregistered = json.load(f)
        print("✓ Coregistered data loaded from saved files")
    else:
        print("\n❌ Cannot proceed without coregistered data.")
        print("Please run the coregistration chunk first.")
        raise SystemExit("Missing prerequisites")

water_masks = {}


NDWI_THRESHOLD = 0.0    # NDWI > 0 = water
S1_THRESHOLD_PERCENTILE = 90  # Top 10% backscatter = likely water (adjustable)

for roi_name, periods in coregistered.items():
    print(f"\n{'='*60}")
    print(f"Processing ROI: {roi_name}")
    print(f"{'='*60}")
   
    water_masks[roi_name] = {}
   
    for period, data in periods.items():
        if not data:
            print(f"  ✗ Missing data for {period}")
            continue
       
        print(f"\n  Period: {period}")
       
        s2_ndwi_path = data['s2_ndwi']
        s1_coreg_path = data['s1_coreg']
        output_dir = f"./output/{roi_name}/water_masks_{period}"
        Path(output_dir).mkdir(parents=True, exist_ok=True)
       
        
        with rasterio.open(s2_ndwi_path) as src:
            ndwi = src.read(1)
            profile = src.profile.copy()
       
        
        with rasterio.open(s1_coreg_path) as src:
            s1_data = src.read(1)
       
        
        s2_mask = (ndwi > NDWI_THRESHOLD).astype(np.uint8)
       
        s2_mask_path = f"{output_dir}/s2_water_mask.tif"
        profile.update(dtype=rasterio.uint8, count=1, nodata=255)
       
        with rasterio.open(s2_mask_path, 'w', **profile) as dst:
            dst.write(s2_mask, 1)
       
        print(f"    ✓ S2 water mask saved: {s2_mask_path}")
       
        
        s1_valid = s1_data[np.isfinite(s1_data)]
        if len(s1_valid) > 0:
            threshold_s1 = np.percentile(s1_valid, S1_THRESHOLD_PERCENTILE)
        else:
            threshold_s1 = 0
       
        s1_mask = (s1_data > threshold_s1).astype(np.uint8)
        combined_mask = ((s2_mask == 1) | (s1_mask == 1)).astype(np.uint8)
       
        combined_mask_path = f"{output_dir}/combined_water_mask.tif"
       
        with rasterio.open(combined_mask_path, 'w', **profile) as dst:
            dst.write(combined_mask, 1)
       
        print(f"    ✓ Combined water mask saved: {combined_mask_path}")
       
        
        water_masks[roi_name][period] = {
            's2_mask': s2_mask,
            'combined_mask': combined_mask,
            's2_water': s2_mask_path,
            'combined_water': combined_mask_path
        }


water_masks_log = {}
for roi_name, periods in water_masks.items():
    water_masks_log[roi_name] = {}
    for period, data in periods.items():
        water_masks_log[roi_name][period] = {
            's2_water': data['s2_water'],
            'combined_water': data['combined_water']
        }

with open('./output/water_masks_log.json', 'w') as f:
    json.dump(water_masks_log, f, indent=2)

print("\n" + "="*60)
print("WATER MASK GENERATION COMPLETE")
print("="*60)
for roi_name, periods in water_masks.items():
    print(f"\n{roi_name}:")
    for period, data in periods.items():
        print(f"  {period}: ✓ Masks generated")
        
print("\n✓ Water masks saved and logged successfully!")

GENERATING WATER MASKS (S2 & COMBINED)

⚠️  ERROR: coregistered not found!
You need to run the coregistration chunk first.

Attempting to load from saved files...
✓ Found coregistration_log.json, loading...
✓ Coregistered data loaded from saved files

Processing ROI: Hirakud_Reservoir

  Period: pre_monsoon
    ✓ S2 water mask saved: ./output/Hirakud_Reservoir/water_masks_pre_monsoon/s2_water_mask.tif
    ✓ Combined water mask saved: ./output/Hirakud_Reservoir/water_masks_pre_monsoon/combined_water_mask.tif

  Period: post_monsoon
    ✓ S2 water mask saved: ./output/Hirakud_Reservoir/water_masks_post_monsoon/s2_water_mask.tif
    ✓ Combined water mask saved: ./output/Hirakud_Reservoir/water_masks_post_monsoon/combined_water_mask.tif

Processing ROI: Chilika_Lake

  Period: pre_monsoon
    ✓ S2 water mask saved: ./output/Chilika_Lake/water_masks_pre_monsoon/s2_water_mask.tif
    ✓ Combined water mask saved: ./output/Chilika_Lake/water_masks_pre_monsoon/combined_water_mask.tif

  Period:

In [6]:
# Create Visualization Maps

print("="*60)
print("CREATING VISUALIZATIONS")
print("="*60)

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

def create_change_visualization(roi_name, change_data, pre_rgb_path, post_rgb_path, output_dir):
    
    with rasterio.open(pre_rgb_path) as src:
        pre_rgb = src.read([1, 2, 3]).transpose(1, 2, 0)
    
    with rasterio.open(post_rgb_path) as src:
        post_rgb = src.read([1, 2, 3]).transpose(1, 2, 0)
    
    change_map = change_data['change_map_array']
    stats = change_data['statistics']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 14))
    fig.suptitle(f'Water Extent Change Detection - {roi_name.replace("_", " ")}', 
                 fontsize=16, fontweight='bold')
    
    ax1 = axes[0, 0]
    ax1.imshow(pre_rgb)
    ax1.set_title('Pre-Monsoon (Sentinel-2 RGB)', fontsize=12, fontweight='bold')
    ax1.axis('off')
    
    ax2 = axes[0, 1]
    ax2.imshow(post_rgb)
    ax2.set_title('Post-Monsoon (Sentinel-2 RGB)', fontsize=12, fontweight='bold')
    ax2.axis('off')
    
    ax3 = axes[1, 0]
    
    colors = ['white', 'blue', 'cyan', 'red']
    cmap = ListedColormap(colors)
    
    im = ax3.imshow(change_map, cmap=cmap, vmin=0, vmax=3)
    ax3.set_title('Water Extent Change Map', fontsize=12, fontweight='bold')
    ax3.axis('off')
    
    legend_elements = [
        mpatches.Patch(color='white', label='No Change (Land)'),
        mpatches.Patch(color='blue', label='Persistent Water'),
        mpatches.Patch(color='cyan', label='Water Gain'),
        mpatches.Patch(color='red', label='Water Loss')
    ]
    ax3.legend(handles=legend_elements, loc='upper right', fontsize=10)
    
    ax4 = axes[1, 1]
    ax4.axis('off')
    
    stats_text = f"""
    CHANGE DETECTION STATISTICS
    {'='*40}
    
    Water Gain:
      • Pixels: {stats['water_gain_pixels']:,}
      • Percentage: {stats['water_gain_pct']:.2f}%
    
    Water Loss:
      • Pixels: {stats['water_loss_pixels']:,}
      • Percentage: {stats['water_loss_pct']:.2f}%
    
    Net Change: {stats['net_change_pct']:+.2f}%
    
    Persistent Water:
      • Pixels: {stats['persistent_water_pixels']:,}
    
    Persistent Land:
      • Pixels: {stats['persistent_land_pixels']:,}
    
    {'='*40}
    Total analyzed pixels: {stats['persistent_water_pixels'] + stats['persistent_land_pixels'] + stats['water_gain_pixels'] + stats['water_loss_pixels']:,}
    """
    
    ax4.text(0.1, 0.5, stats_text, fontsize=11, family='monospace',
             verticalalignment='center', transform=ax4.transAxes)
    
    plt.tight_layout()
    
    output_path = f"{output_dir}/{roi_name}_change_visualization.png"
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  ✓ Saved: {output_path}")
    
    return output_path

def create_side_by_side_comparison(roi_name, pre_mask, post_mask, output_dir):
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle(f'Water Mask Comparison - {roi_name.replace("_", " ")}', 
                 fontsize=14, fontweight='bold')
    
    ax1 = axes[0]
    ax1.imshow(pre_mask, cmap='Blues', vmin=0, vmax=1)
    ax1.set_title('Pre-Monsoon Water Extent', fontsize=12)
    ax1.axis('off')
    
    ax2 = axes[1]
    ax2.imshow(post_mask, cmap='Blues', vmin=0, vmax=1)
    ax2.set_title('Post-Monsoon Water Extent', fontsize=12)
    ax2.axis('off')
    
    plt.tight_layout()
    
    output_path = f"{output_dir}/{roi_name}_water_comparison.png"
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  ✓ Saved: {output_path}")
    
    return output_path

visualization_outputs = {}

for roi_name in change_results.keys():
    print(f"\n{'='*60}")
    print(f"Creating visualizations: {roi_name}")
    print(f"{'='*60}")
    
    output_dir = f"./output/{roi_name}/visualizations"
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    pre_data = water_masks[roi_name]['pre_monsoon']
    post_data = water_masks[roi_name]['post_monsoon']
    change_data = change_results[roi_name]['s2']
    
    pre_rgb = coregistered[roi_name]['pre_monsoon']['s2_rgb']
    post_rgb = coregistered[roi_name]['post_monsoon']['s2_rgb']
    
    print("\n  Creating change detection visualization...")
    vis_path = create_change_visualization(
        roi_name, 
        change_data,
        pre_rgb,
        post_rgb,
        output_dir
    )
    
    print("\n  Creating water mask comparison...")
    comp_path = create_side_by_side_comparison(
        roi_name,
        pre_data['s2_mask'],
        post_data['s2_mask'],
        output_dir
    )
    
    visualization_outputs[roi_name] = {
        'change_visualization': vis_path,
        'water_comparison': comp_path
    }

print("\n" + "="*60)
print("VISUALIZATION COMPLETE")
print("="*60)

for roi_name, paths in visualization_outputs.items():
    print(f"\n{roi_name}:")
    print(f"  Change visualization: {paths['change_visualization']}")
    print(f"  Water comparison: {paths['water_comparison']}")

print("\n✓ All visualizations saved successfully!")

CREATING VISUALIZATIONS

Creating visualizations: Hirakud_Reservoir

  Creating change detection visualization...
  ✓ Saved: ./output/Hirakud_Reservoir/visualizations/Hirakud_Reservoir_change_visualization.png

  Creating water mask comparison...
  ✓ Saved: ./output/Hirakud_Reservoir/visualizations/Hirakud_Reservoir_water_comparison.png

Creating visualizations: Chilika_Lake

  Creating change detection visualization...
  ✓ Saved: ./output/Chilika_Lake/visualizations/Chilika_Lake_change_visualization.png

  Creating water mask comparison...
  ✓ Saved: ./output/Chilika_Lake/visualizations/Chilika_Lake_water_comparison.png

VISUALIZATION COMPLETE

Hirakud_Reservoir:
  Change visualization: ./output/Hirakud_Reservoir/visualizations/Hirakud_Reservoir_change_visualization.png
  Water comparison: ./output/Hirakud_Reservoir/visualizations/Hirakud_Reservoir_water_comparison.png

Chilika_Lake:
  Change visualization: ./output/Chilika_Lake/visualizations/Chilika_Lake_change_visualization.png
  W